# <center>Interactive Data Visualization in Python With Bokeh</center>

## Adding Interaction

In [48]:
!pip install pyproj
from bokeh.io import curdoc
from pyproj import Proj, transform
import pandas as pd
import datetime as dt
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row, gridplot
from bokeh.models import Slider, Select
from bokeh.models import DatePicker, Select, ColumnDataSource, ColorBar
import pandas as pd
from bokeh.io import curdoc
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row, gridplot
from bokeh.models import Slider, Select
from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM

import warnings

In [49]:
data = pd.read_csv("owid-covid-data.csv")
data.set_index('date', inplace=True)

data.head(5)

,iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-02-24,AFG,Asia,Afghanistan,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-25,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-26,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-27,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-28,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [50]:
# Make a list of the unique values from the region column: regions_list
regions_list = data.location.unique().tolist()

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

In [51]:
#define beberapa variabel untuk membuat peta dengan cartodbpositron
inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')

ind_lon1, ind_lat1 = transform(outProj,inProj,90,-15)
ind_lon2, ind_lat2 = transform(outProj,inProj,150,20)
cartodb = get_provider(CARTODBPOSITRON)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  """
/usr/local/lib/

In [52]:
df = data[data.index == '2020-02-24']

#define variabel nam untuk menampung nama kolom yang di select
nam = []
for i in df.new_cases:
    nam.append("new_cases")
   # nam.append("total_death")

#source digunakan untuk menampilkan data yang akan ditampilkan (data awal)
source = ColumnDataSource(data={
    'x'         : df.total_cases, #define x dengan kolom mercatorX dari data dengan index tanggal 2020-03-01
    'y'         : df.total_deaths, #define y dengan kolom mercatorY dari data dengan index tanggal 2020-03-01
    'dat'       : df.new_cases, #define dat dengan kolom new_cases dari data dengan index tanggal 2020-03-01
    'nama'      : df.location #define nama dengan nama kolom new_cases 
})

In [53]:
#mapper adalah list color dimana akan berwarna merah jika nilai dari data sekitar 800000 dan berwarna biru jika bernilai mendekati 0
mapper = linear_cmap('dat', Spectral6 , 0 , 849875)

#menampilkan peta pada visualisasi data
plot = figure(plot_width=900, plot_height=700,
           x_range=(ind_lon1, ind_lon2), y_range=(ind_lat1, ind_lat2),
           x_axis_type="mercator", y_axis_type="mercator",
           tooltips=[
                    ("lokasi", "@nama"), ("Jumlah", "@dat"), 
                    ("total kematian", "@x"), ("total_kasus", "@y") #menampilkan data tiap kolom/data yang diselect
                    ],
           title="Covid 19")

plot.add_tile(cartodb)
#plotting scatter plot (circle)
plot.circle(x='x', y='y',
         size=10,
         line_color=mapper, color=mapper,
         fill_alpha=1.0,
         source=source)
#menampilkan color bar
color_bar = ColorBar(color_mapper=mapper['transform'], width=5)

plot.add_layout(color_bar, 'right')


In [54]:
def update_plot(attr, old, new):
    df = data[data.index == str(dPicker.value)] #update 'df' dengan data dari index date yang di select oleh fitur datepicker
    nam = []
    for i in df.new_cases:
        nam.append(str(data_select.value)) #update var nam
        #nam.append(str(data_select1.value))
    source.data = {
        'x'         : df.total_cases, #update x dengan kolom mercatorX dari data index date yang di select oleh fitur datepicker
        'y'         : df.total_deaths, #define y dengan kolom mercatorY dari data index date yang di select oleh fitur datepicker
        'dat'       : df[data_select.value], #update dat dengan kolom new_cases dari data kolom yang diselect pada fitur dropdwon select
        'nama'      : nam #update nama dengan nama kolom sesuai dengan kolom yang diselect
    }

In [55]:
#define fitur interaktif date picker
dPicker = DatePicker(
    title = 'Date',
    value=dt.datetime(2020, 2, 24).date(), 
    min_date= dt.datetime(2020, 2, 24).date(), max_date=dt.datetime(2021, 12, 3).date()
)

dPicker.on_change('value', update_plot)

#define fitur interaktif dropdown dan select
data_select = Select(
    options=['total_cases', 'new_cases', 'total_deaths'	],
    value='new_cases',
    title='x-axis data'
)

data_select.on_change('value', update_plot)

#data_select1 = Select(
 #   options=['new_cases', 'new_deaths',	'new_recovered', 'new_activeCases', 'total_cases', 'total_deaths',	'total_recovered', 'total_activeCases'],
  #  value='total_death',
   # title='y-axis data'
#)

#data_select1.on_change('value', update_plot)

#memasukan seluruh fitur interaktif dan juga plotingan kedalam layout
layout = row(widgetbox(dPicker, data_select), plot) 
curdoc().add_root(layout)

For more on all things interaction in Bokeh, [**Adding Interactions**](https://docs.bokeh.org/en/latest/docs/user_guide/interaction.html) in the Bokeh User Guide is a great place to start.